# YouTube Summarizer
### Working with small LangChain Agent Systems
In this notebook, we are going to create a langchain agent that will summarize a video when passed a YouTube URL.

In [ ]:
!pip install openai langchain langchain-openai youtube-transcript-api

### Initial Experimentation
Below, I'm starting to mess around with just grabbing the raw text from a youtube video

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = '_fe0Acsr33M'
raw_transcript = YouTubeTranscriptApi.get_transcript(video_id)


In [ ]:
import re

full_text = " ".join([item["text"] for item in raw_transcript])
clean_text = re.sub(r'\[.*?\]', '', full_text)

print(clean_text)

## Next Steps
Since that works, I'm now going to set up my AI envrionment with OpenAI access and Langchain tools to get an agentic workflow going

In [ ]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("OpenAI API Key:")

encoder = OpenAIEncoder(name="text-embedding-3-small")

In [ ]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
  input: str
  chat_history: list[BaseMessage]
  intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]



## Now we start building the actual tools we will use in the workflow

In [ ]:
from langchain_core.tools import tool

@tool("fetch_transcript")
def fetch_transcript(url: str) -> Union[str, ValueError] :
  """Gets the transcript from a Youtube video, when passed the video's URL"""

  match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11})', url)
  video_id = match.group(1)

  if not video_id:
    return ValueError("No video matches that URL")

  raw_transcript = YouTubeTranscriptApi.get_transcript(video_id)
  full_text = " ".join([item["text"] for item in raw_transcript])
  transcript = re.sub(r'\[.*?\]', '', full_text)

  return transcript

To test the tool, you can run .invoke. You can also verify your parameters by printing out .name, .args, and .description

In [ ]:
target_url = 'https://www.youtube.com/watch?v=_fe0Acsr33M'
print(
    f'{fetch_transcript.name} /n {fetch_transcript.args} \n {fetch_transcript.description}'
    # fetch_transcript.invoke(input={"url": 'https://www.youtube.com/watch?v=_fe0Acsr33M'})
)

In [ ]:
print(fetch_transcript.invoke(input={"url": target_url}))

## Build the Chain to use your tools
Now we can start setting up the LLM to use our tools and create the langchain chain. First we hardcode test to see if the agent will run the tool like we want.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate


llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([fetch_transcript])

In [ ]:
msg = llm_with_tools.invoke(f'what does dave talk about in {target_url}?')
msg.tool_calls

Since that all looks good, we build up our prompt. It's critical to have chat_history and agent_scratchpad as placeholders due to back-end requirements for langchain agents. The system message can be anything.

In [ ]:
system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=[],
        input_types={},
        partial_variables={},
        template='You are a helpful assistant that summarizes YouTube videos.'
    ),
    additional_kwargs={}
)

human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=['input'],
        input_types={},
        partial_variables={},
        template='{input}'
    ),
    additional_kwargs={}
)

# Create the ChatPromptTemplate with placeholders for chat history and agent scratchpad
prompt = ChatPromptTemplate.from_messages([
    system_message,
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    human_message,
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

# Define tools and create the agent and agent executor
tools = [fetch_transcript]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


Let's run it!

In [ ]:
user_prompt = f'what does dave talk about in {target_url}?'

agent_executor.invoke(
    {
        "input": user_prompt
    }
)